In [1]:
pip install shared_utils

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

import shared_utils
import pandas as pd
import geopandas as gpd
from calitp_data_analysis.sql import get_engine
from shared_utils import gtfs_utils_v2
db_engine = get_engine()

import gcsfs
from calitp_data_analysis import get_fs
from calitp_data_analysis import geography_utils, utils
fs = get_fs()
import re
import google.auth
import os
credentials, project = google.auth.default()


In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

In [5]:
import datetime as dt

In [6]:
# set date parameters
analysis_wkd = dt.date(2022,11,30)
analysis_sat = dt.date(2022,12,3)
analysis_sun = dt.date(2022,12,4)

In [7]:
def read_parquet_from_gcs(filename):
    gcs_path = GCS_FILE_PATH.replace("gs://", "") + filename
    with fs.open(gcs_path, 'rb') as f:
        return gpd.read_parquet(f)

In [8]:
acs_ca = read_parquet_from_gcs("tracts_ca_acs.parquet")

In [9]:
jobdata = pd.read_parquet(f"{GCS_FILE_PATH}/job_density_2022.parquet")[['GEOID', 'jobs_tot']]

In [10]:
# Inner join with ACS data on 'geo_id'
acs_ca = acs_ca.merge(jobdata, on = 'GEOID', how='left')

In [11]:
acs_ca = acs_ca.to_crs(epsg=3347)

In [13]:
with db_engine.connect() as connection:
    query = """
        SELECT
            key, name, ntd_id, ntd_agency_info_key
        FROM 
            cal-itp-data-infra.mart_transit_database.dim_organizations
    """
    organizations= pd.read_sql(query, connection)

In [ ]:
organizations_filtered = organizations.dropna(subset=["ntd_id", "ntd_agency_info_key"], how="all")

In [ ]:
organizations_filtered.drop_duplicates(inplace=True)

In [ ]:
organizations_filtered.shape

In [ ]:
with db_engine.connect() as connection:
    query = """
        SELECT
            gtfs_dataset_key, organization_key, gtfs_dataset_name
        FROM 
            cal-itp-data-infra.mart_transit_database.bridge_organizations_x_gtfs_datasets_produced
    """
    organizations_gtfs_key= pd.read_sql(query, connection)

In [ ]:
organizations_gtfs_key.shape

In [ ]:
organizations_gtfs_key.head(5)

In [ ]:
organizations_gtfs_key = organizations_gtfs_key.dropna(
    subset=["organization_key", "gtfs_dataset_key"]
)


In [ ]:
organizations_itp_ids = pd.merge(
    organizations_filtered,
    organizations_gtfs_key,
    how="inner",
    left_on="key",
    right_on="organization_key",
)


In [ ]:
organizations_itp_ids = (
    organizations_itp_ids
    .drop_duplicates(subset=["name", "ntd_id"])
    .reset_index(drop=True)
)


In [51]:
organizations_itp_ids.head(5)

,key,name,ntd_id,ntd_agency_info_key,gtfs_dataset_key,organization_key,gtfs_dataset_name
0,ba7f7c075f7b127cb0e8ba7f487016c4,Alameda-Contra Costa Transit District,90014,rec0zt7fBmP2s3F3g,c9e7bedaaf50176c71d1cc267a5171a1,ba7f7c075f7b127cb0e8ba7f487016c4,AC Transit Schedule
1,f45204da25a4f2a031582a0d0bae0b18,Amador Regional Transit System,9R02-91000,recyQ9Dp6JKnr3Lmr,e1099a4938e0279f09d5bbc10a1702a4,f45204da25a4f2a031582a0d0bae0b18,Amador Schedule
2,17cbc21bfd085d6f4176cdebd9104995,Anaheim Transportation Network,90211,rec6g6iKaKu4n4r89,a89252857dd2ba8e9b701f1819cae364,17cbc21bfd085d6f4176cdebd9104995,Anaheim Resort Alerts
3,4578b78ef0af675194e838a0e61d6b0c,Antelope Valley Transit Authority,90121,recTmzDLUS5kOkdqI,8bb9a39aaf1fb807ce855b26708155f5,4578b78ef0af675194e838a0e61d6b0c,Antelope Valley Transit Authority Alerts
4,92d8fe341ae7f041b2e17665430dce7c,Butte County Association of Governments,90208,rec7wacWpNhcSlyV3,6759e4b8ac7ec68488e0a78f3311c71e,92d8fe341ae7f041b2e17665430dce7c,B-Line Schedule


In [50]:
organizations_itp_ids.shape

(143, 7)

In [39]:
schedule_gtfs_dataset_keys = (
    organizations_itp_ids["gtfs_dataset_key"]
    .unique()
    .tolist()
)

In [43]:
len(schedule_gtfs_dataset_keys)

140

In [25]:
def daily_trips(day_type, analysis_dt, schedule_key):
    # Get all feeds for that analysis date
    feeds_for_day = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=analysis_dt)

    # Only keep feeds where gtfs_dataset_key is in our known keys
    feeds_for_day = feeds_for_day[
        feeds_for_day["gtfs_dataset_key"].isin(schedule_gtfs_dataset_keys)
    ]

    # Then filter for the specific schedule_key
    feed_keys = feeds_for_day[
        feeds_for_day["gtfs_dataset_key"] == schedule_key
    ]["feed_key"].unique().tolist()

    if feed_keys:
        print(f"[{day_type}] schedule_key: {schedule_key}, feed_keys found: {feed_keys}")
    else:
        # no print here if empty
        return pd.DataFrame()

    # Get trips data using feed_keys
    trips_df = gtfs_utils_v2.get_trips(
        selected_date=analysis_dt,
        operator_feeds=feed_keys,
        trip_cols=[
            "feed_key", "trip_id", "trip_instance_key", "route_id", "shape_id",
            "direction_id", "route_type", "route_short_name"
        ]
    ).drop_duplicates()
    
    trips_df["route_name_id"] = (
        trips_df["route_short_name"].fillna("") + "_" + trips_df["route_id"]
    )

    print(f"[{day_type}] schedule_key: {schedule_key}, trips found: {len(trips_df)}")
    
    if trips_df.empty:
        return pd.DataFrame()


    # Get stop times
    stoptimes_df = gtfs_utils_v2.get_stop_times(
        selected_date=analysis_dt,
        operator_feeds=feed_keys,
        stop_time_cols=["trip_id", "feed_key", "stop_id", "arrival_sec", "departure_sec"],
        trip_df=trips_df,      
        get_df=True
    )

    
    print(f"[{day_type}] schedule_key: {schedule_key}, stop times found: {len(stoptimes_df)}")

    # Merge with trip data
    stoptimes_df = stoptimes_df.merge(trips_df, on=["trip_id", "feed_key"])

    # Group and summarize
    grouped = stoptimes_df.groupby(["feed_key", "route_type", "stop_id"])

    summary = grouped.agg(
        n_trips=("trip_instance_key", pd.Series.nunique),
        n_routes=("route_id", pd.Series.nunique),
        route_list=("route_name_id", lambda x: [x.tolist()])
    ).reset_index()

    summary["daytype"] = day_type
    summary["analysis_date"] = analysis_dt
    summary["route_list_string"] = summary["route_list"].apply(lambda x: ",".join(x[0]))

    # Get stop locations
    stops_geo = gtfs_utils_v2.get_stops(
        selected_date=analysis_dt,
        operator_feeds=feed_keys,
        stop_cols=["feed_key", "stop_id", "stop_desc", "stop_name"],
        get_df=True
    )
    


    stoptimes_geo = stops_geo.merge(summary, on=["feed_key", "stop_id"])

    # Spatial processing
    stoptimes_geo = stoptimes_geo.to_crs(epsg=3347)
    stoptimes_geo["geometry"] = stoptimes_geo.buffer(402.336)  # 0.25 miles

    # Join to ACS
    stops_acs_joined = stoptimes_geo.sjoin(acs_ca, how="left", predicate="intersects")

    group_cols = [
        "feed_key", "stop_id", "stop_name", "n_trips", "n_routes",
        "daytype", "analysis_date", "route_list_string"
    ]

    acs_summary = stops_acs_joined.groupby(group_cols).agg(
        sum_tracts=("GEOID", pd.Series.nunique),
        sum_total_pop=("total_pop", "sum"),
        sum_households=("households", "sum"),
        sum_not_us_citizen_pop=("not_us_citizen_pop", "sum"),
        sum_youth_pop=("youth_pop", "sum"),
        sum_seniors_pop=("seniors_pop", "sum"),
        sum_pop_determined_poverty_status=("pop_determined_poverty_status", "sum"),
        sum_poverty=("poverty", "sum"),
        sum_no_car=("workers_with_no_car", "sum"),
        sum_no_cars=("households_with_no_cars", "sum"),
        sum_land_area=("ALAND", "sum"),
        sum_jobs=("jobs_tot", "sum")
    ).reset_index()

    # Derived metrics
    acs_summary["land_area_sqkm"] = acs_summary["sum_land_area"] / 1_000_000
    acs_summary["pop_density"] = acs_summary["sum_total_pop"] / acs_summary["land_area_sqkm"]
    acs_summary["job_density"] = acs_summary["sum_jobs"] / acs_summary["land_area_sqkm"]
    acs_summary["pct_not_us_citizen_pop"] = (acs_summary["sum_not_us_citizen_pop"] / acs_summary["sum_total_pop"]) * 100
    acs_summary["pct_youth_pop"] = (acs_summary["sum_youth_pop"] / acs_summary["sum_total_pop"]) * 100
    acs_summary["pct_seniors_pop"] = (acs_summary["sum_seniors_pop"] / acs_summary["sum_total_pop"]) * 100
    acs_summary["pct_poverty"] = (acs_summary["sum_poverty"] / acs_summary["sum_pop_determined_poverty_status"]) * 100
    acs_summary["pct_pop_workers_no_car"] = (acs_summary["sum_no_car"] / acs_summary["sum_total_pop"]) * 100
    acs_summary["pct_hh_no_cars"] = (acs_summary["sum_no_cars"] / acs_summary["sum_households"]) * 100

    # Final join
    acs_summary["gtfs_dataset_key"] = schedule_key
    acs_summary = acs_summary.merge(
        organizations_itp_ids,
        how="left",
        on="gtfs_dataset_key"
    ).drop_duplicates()

    return acs_summary

In [26]:
fs_list = fs.ls(f"{GCS_FILE_PATH}tool_data_2025/")

In [27]:
ran_operators = []
for path in fs_list:
    try:
        if path.endswith(".parquet"):
            key = int(path.split("tool_data_2025/")[1].split("_")[2].split(".")[0])
            ran_operators.append(key)
    except (IndexError, ValueError):
        continue

In [28]:
for schedule_key in schedule_gtfs_dataset_keys:
    if schedule_key in ran_operators:
        print(f"already ran: {schedule_key}")
        continue

    print(f"processing schedule_key {schedule_key}")

    stops_weekday = daily_trips("Weekday", analysis_wkd, schedule_key)
    stops_saturday = daily_trips("Saturday", analysis_sat, schedule_key)
    stops_sunday = daily_trips("Sunday", analysis_sun, schedule_key)

    stoptimes_all = pd.concat(
        [stops_weekday, stops_saturday, stops_sunday],
        ignore_index=True
    )

    if not stoptimes_all.empty:
        output_path = f"{GCS_FILE_PATH}tool_data_2025/trips_perstop_{schedule_key}.parquet"
        stoptimes_all.to_parquet(output_path, compression="snappy")
        print(f"Saved results to {output_path}")
    else:
        print(f"No data for {schedule_key}")



processing schedule_key c9e7bedaaf50176c71d1cc267a5171a1
No data for c9e7bedaaf50176c71d1cc267a5171a1
processing schedule_key e1099a4938e0279f09d5bbc10a1702a4
[Weekday] schedule_key: e1099a4938e0279f09d5bbc10a1702a4, feed_keys found: ['ef1f0da66bb00384c69f8102a1f9ae7a']
could not get metrolink feed on 2022-11-30!
[Weekday] schedule_key: e1099a4938e0279f09d5bbc10a1702a4, trips found: 29


/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/lib/python3.11/site-packages/pandas/core

[Weekday] schedule_key: e1099a4938e0279f09d5bbc10a1702a4, stop times found: 668
[Saturday] schedule_key: e1099a4938e0279f09d5bbc10a1702a4, feed_keys found: ['ef1f0da66bb00384c69f8102a1f9ae7a']
could not get metrolink feed on 2022-12-03!
[Saturday] schedule_key: e1099a4938e0279f09d5bbc10a1702a4, trips found: 0
[Sunday] schedule_key: e1099a4938e0279f09d5bbc10a1702a4, feed_keys found: ['ef1f0da66bb00384c69f8102a1f9ae7a']
could not get metrolink feed on 2022-12-04!
[Sunday] schedule_key: e1099a4938e0279f09d5bbc10a1702a4, trips found: 0
Saved results to gs://calitp-analytics-data/data-analyses/ahsc_grant/tool_data_2025/trips_perstop_e1099a4938e0279f09d5bbc10a1702a4.parquet
processing schedule_key a89252857dd2ba8e9b701f1819cae364
No data for a89252857dd2ba8e9b701f1819cae364
processing schedule_key 8bb9a39aaf1fb807ce855b26708155f5
No data for 8bb9a39aaf1fb807ce855b26708155f5
processing schedule_key 6759e4b8ac7ec68488e0a78f3311c71e
[Weekday] schedule_key: 6759e4b8ac7ec68488e0a78f3311c71e, feed

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(


[Weekday] schedule_key: 6759e4b8ac7ec68488e0a78f3311c71e, stop times found: 9394


...

[Weekday] schedule_key: e33f30cf19564013bf2f80bfd2834574, stop times found: 5526


/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


[Saturday] schedule_key: e33f30cf19564013bf2f80bfd2834574, feed_keys found: ['03f30ee5933125591a265f63fc8f7d66']
could not get metrolink feed on 2022-12-03!
[Saturday] schedule_key: e33f30cf19564013bf2f80bfd2834574, trips found: 56


/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/lib/python3.11/site-packages/pandas/core

[Saturday] schedule_key: e33f30cf19564013bf2f80bfd2834574, stop times found: 2132
[Sunday] schedule_key: e33f30cf19564013bf2f80bfd2834574, feed_keys found: ['03f30ee5933125591a265f63fc8f7d66']
could not get metrolink feed on 2022-12-04!
[Sunday] schedule_key: e33f30cf19564013bf2f80bfd2834574, trips found: 0
Saved results to gs://calitp-analytics-data/data-analyses/ahsc_grant/tool_data_2025/trips_perstop_e33f30cf19564013bf2f80bfd2834574.parquet
processing schedule_key e47aa2f88b679192f9381b076b561bcf
No data for e47aa2f88b679192f9381b076b561bcf
processing schedule_key c872474db7f5825b5f360ddad4361331
No data for c872474db7f5825b5f360ddad4361331
processing schedule_key 47cd9b06cc79bf651578b12b4ce7bb20
No data for 47cd9b06cc79bf651578b12b4ce7bb20
processing schedule_key b0760015c9fcd0500c4fddd5b9bb115b
No data for b0760015c9fcd0500c4fddd5b9bb115b
processing schedule_key ebd7c3783136764d4f1b8da3e613f77b
No data for ebd7c3783136764d4f1b8da3e613f77b
processing schedule_key b6dd4c18de23dfb637